In [1]:
from bitcoinlib.keys import Key, sign, verify
from collections import defaultdict
import hashlib
import os

# Blockchain structure
# [[previous_hashblock, hashblock, transactions, admin_sign], next_block]
# transactions = [{amount, type_transaction, user, user_sign, user_destinatary (optional)}, next_transaction]
# type_transactions = create, transfer

def check_blocks(blocks, admin_pk):
    users_history = defaultdict(float)

    if blocks[0][0] != "000000000000000":
        print("Error: The initial block does not have a valid hash.")
        return False

    for i, current_block in enumerate(blocks):
        if i > 0 and current_block[0] != blocks[i - 1][1]:
            print(f"Error: Invalid link between block {i - 1} and block {i}.")
            return False
        
        block_data = current_block[0] + "".join(
            [
                f"{tx['amount']}{tx['type_transaction']}{tx['user']}"
                + (str(tx.get("user_destinatary", "")) if tx['type_transaction'] == "transfer" else "")
                for tx in current_block[2]
            ]
        )
        block_hash = hashlib.sha256(block_data.encode('utf-8')).hexdigest()
        if not verify(block_hash, current_block[3], admin_pk):
            print(f"Error: Invalid administrator signature on block {i}.")
            return False

        for transaction in current_block[2]:
            if "amount" not in transaction or "type_transaction" not in transaction or "user" not in transaction:
                print(f"Error: Missing fields in a transaction in block {i}.")
                return False

            tx_data = f"{transaction['amount']}{transaction['type_transaction']}{transaction['user']}"
            if transaction["type_transaction"] == "transfer":
                if "user_destinatary" not in transaction:
                    print(f"Error: Missing 'user_destinatary' in transfer transaction in block {i}.")
                    return False
                tx_data += str(transaction["user_destinatary"])

            computed_hash = hashlib.sha256((current_block[0] + tx_data).encode('utf-8')).hexdigest()
            if not verify(computed_hash, transaction["user_sign"], transaction["user"]):
                print(f"Error: Invalid user signature in a transaction in block {i}.")
                return False

            if transaction["type_transaction"] == "create":
                users_history[transaction["user"]] += transaction["amount"]
            elif transaction["type_transaction"] == "transfer":
                users_history[transaction["user"]] -= transaction["amount"]
                users_history[transaction["user_destinatary"]] += transaction["amount"]

                if users_history[transaction["user"]] < 0:
                    print(f"Error: Insufficient balance for user {transaction['user']} in block {i}.")
                    return False

    return users_history


def create_block(transactions, admin_sk, admin_pk, blocks=[]):
    if not transactions:
        print("Error: No transactions provided to create a block.")
        return False

    block = [None, None, [], None]
    block[0] = "000000000000000" if not blocks else blocks[-1][1]

    for tx in transactions:
        if "amount" not in tx or "type_transaction" not in tx or "user" not in tx:
            print("Error: Missing transaction fields.")
            return False

        tx_data = f"{tx['amount']}{tx['type_transaction']}{tx['user']}"
        if tx["type_transaction"] == "transfer" and "user_destinatary" in tx:
            tx_data += str(tx["user_destinatary"])
        elif tx["type_transaction"] == "transfer":
            print("Error: Transfer transaction missing 'user_destinatary'.")
            return False

        tx_hash = hashlib.sha256((block[0] + tx_data).encode('utf-8')).hexdigest()
        tx["user_sign"] = sign(tx_hash, tx["user_sk"])
        block[2].append(tx)

    block_data = block[0] + "".join(
        [
            f"{tx['amount']}{tx['type_transaction']}{tx['user']}"
            + (str(tx.get("user_destinatary", "")) if tx['type_transaction'] == "transfer" else "")
            for tx in block[2]
        ]
    )
    block[1] = hashlib.sha256(block_data.encode('utf-8')).hexdigest()
    block[3] = sign(block[1], admin_sk)

    blocks.append(block)
    if check_blocks(blocks, admin_pk):
        return blocks
    else:
        print("Error: Block validation failed after adding the new block.")
        return False


def save_block_to_file(block, block_index, filename, admin_sign):
    with open(filename, "w") as f:
        f.write(f"Block {block_index}:\n")
        f.write(f"Previous hash = {block[0]}\n")
        f.write(f"Current hash = {block[1]}\n")
        for tx in block[2]:
            if tx["type_transaction"] == "create":
                f.write(f"{tx['amount']} coins are created for {tx['user']}\n")
            elif tx["type_transaction"] == "transfer":
                f.write(f"{tx['user']} sends {tx['amount']} coins to {tx['user_destinatary']} (Signature: {tx['user_sign']})\n")
        f.write(f"(Admin Signature: {admin_sign})\n")
        f.write("----------\n")



def publish_blocks(blocks):
    os.makedirs("blocks", exist_ok=True)
    for i, block in enumerate(blocks):
        filename = f"blocks/block_{i}.txt"
        save_block_to_file(block, i, filename, str(block[3]))
        print(f"Block {i} saved in: {filename}")


def print_blocks(blocks):
    for i, block in enumerate(blocks):
        print(f"Block {i} --------------------------------------------------------------------------------------------")
        print("Previous hash:", block[0])
        print("Current hash:", block[1])
        for j, tx in enumerate(block[2]):
            print(f"  Transaction {j + 1}:")
            for key, value in tx.items():
                print(f"    {key}: {value}")


In [2]:
from bitcoinlib.keys import Key
admin_k = Key()
admin_sk, admin_pk = admin_k.secret, admin_k.public()
user1_k = Key()
user1_sk, user1_pk = user1_k.secret, user1_k.public()
user2_k = Key()
user2_sk, user2_pk = user2_k.secret, user2_k.public()
user3_k = Key()
user3_sk, user3_pk = user3_k.secret, user3_k.public()

blocks = []

transactions_block_0 = [
    {"amount": 10, "type_transaction": "create", "user": user1_pk, "user_sk": user1_sk},
]

transactions_block_1 = [
    {"amount": 3, "type_transaction": "transfer", "user": user1_pk, "user_sk": user1_sk, "user_destinatary": user2_pk},
]

transactions_block_2 = [
    {"amount": 4, "type_transaction": "transfer", "user": user1_pk, "user_sk": user1_sk, "user_destinatary": user3_pk},
    {"amount": 2, "type_transaction": "transfer", "user": user2_pk, "user_sk": user2_sk, "user_destinatary": user3_pk},
]

blocks = create_block(transactions_block_0, admin_sk, admin_pk, blocks=blocks)
blocks = create_block(transactions_block_1, admin_sk, admin_pk, blocks=blocks)
blocks = create_block(transactions_block_2, admin_sk, admin_pk, blocks=blocks)

publish_blocks(blocks)

print_blocks(blocks)

Block 0 saved in: blocks/block_0.txt
Block 1 saved in: blocks/block_1.txt
Block 2 saved in: blocks/block_2.txt
Block 0 --------------------------------------------------------------------------------------------
Previous hash: 000000000000000
Current hash: c33c66cf0af20d4490db7c6a180acbdcacb3d0b663ab10a7d0d8ab86b6f92466
  Transaction 1:
    amount: 10
    type_transaction: create
    user: 03a978c0defcb5c8fa3a59c5a2e04b89715d9c637bdf87862f9bf513fd585eec83
    user_sk: 1761151804122849308087061389323461476791447700310129058774240389823911842835
    user_sign: 30440220506968be2a021c33eb4ba6d019331b3665f3038c92bb0ce6ad03c24c973f375e0220498e026a50e30cbf7ed074b096f4d69d18a036718f20c975e6ac48a1009f518e01
Block 1 --------------------------------------------------------------------------------------------
Previous hash: c33c66cf0af20d4490db7c6a180acbdcacb3d0b663ab10a7d0d8ab86b6f92466
Current hash: 2f14d5c547022611b9ea02c8d651d44eceec177ecf146008582f5a5728da8791
  Transaction 1:
    amount: 3
 